In [1]:
import sys
sys.path.append("..")

import plotly.express as px
import plotly.io as pio
from history_parser import HistoryParser
import pandas as pd
import numpy as np

import plotly.graph_objects as go

# Create a custom template
custom_theme = go.layout.Template(
    layout=go.Layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',  # Transparent plot background
        paper_bgcolor='rgba(10, 10, 10, 1)',  # Dark paper background
        font_color='white',  # White font for visibility
        title_font_size=20,
        legend_title_font_size=16,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            title_font_size=16,
            showgrid=True,
            gridwidth=1,
            gridcolor='DimGray'
        ),
        yaxis=dict(
            title_font_size=16,
            showgrid=True,
            gridwidth=1,
            gridcolor='DimGray',
            autorange=True
        )
    )
)

pio.templates["custom_dark_theme"] = custom_theme
pio.templates.default = "custom_dark_theme"


In [18]:
from hand_parser import HandParser


history = HistoryParser().parse_file("../../games/3.9/3.9.csv")
hand_history = HandParser().parse_folder("../../games/3.9/hand_data/")

In [20]:
hand_history[152]

{Player(username='yyj', uid='UXaWQPJt1x'): (Card(rank=12, suit=<CardSuit.DIAMONDS: '♦'>),
  Card(rank=5, suit=<CardSuit.CLUBS: '♣'>)),
 Player(username='Jonathan', uid='ArgKs65s5n'): (Card(rank=4, suit=<CardSuit.SPADES: '♠'>),
  Card(rank=6, suit=<CardSuit.HEARTS: '♥'>)),
 Player(username='az', uid='_c6_FOIS6W'): (Card(rank=9, suit=<CardSuit.DIAMONDS: '♦'>),
  Card(rank=4, suit=<CardSuit.DIAMONDS: '♦'>)),
 Player(username='michael', uid='UFkn4-rw65'): (Card(rank=7, suit=<CardSuit.HEARTS: '♥'>),
  Card(rank=13, suit=<CardSuit.HEARTS: '♥'>))}

In [39]:
from structs import Card, Player, PlyState
from actions import Raise

dealer_opens: dict[Player, list[tuple[Card, Card]]]= {}

for cards, ply in zip(hand_history, history):
	for action in ply.actions[PlyState.PREFLOP]:
		if isinstance(action[1], Raise) and action[0] in cards:
			if action[0] not in dealer_opens:
				dealer_opens[action[0]] = []
			dealer_opens[action[0]].append(cards[action[0]])
			break


In [40]:
import plotly.figure_factory as ff
import plotly.graph_objects as go

from structs import Card

ranks = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

def plot_range(cards: list[tuple[Card, Card]], title="") -> None:
    def get_label(r1, r2):
        suited = False
        if(r2 > r1):
            r1, r2 = r2, r1
            suited = True
        return f'{ranks[r2]}{ranks[r1]}{"s" if suited else ("o" if r1 != r2 else "")}'

    heatmap_values = [[0 for _ in range(13)] for _ in range(13)]
    string_matrix = [[get_label(12-i,j) for j in range(13)] for i in range(13)]

    for hole in cards:
        c1, c2 = hole
        if(c1.suit != c2.suit):
            heatmap_values[c1.rank-2][(14-c2.rank)%13] += 1
        else:
            heatmap_values[c2.rank-2][(14-c1.rank)%13] += 1

    fig = ff.create_annotated_heatmap(
        z=heatmap_values,
        annotation_text=string_matrix,
        colorscale='blues'
    )

    # Adjusting the layout to attempt square cells
    # Note: These dimensions might need adjustment based on your display or specific needs
    fig.update_layout(
        width=650,  # Adjust the figure width
        height=650,  # Adjust the figure height to match the width for square cells
        autosize=False,
        xaxis=dict(constrain='domain'),  # This can help maintain proportions
        yaxis=dict(scaleanchor="x", scaleratio=1),  # This forces the y-axis to scale with the x-axis
    )

    # Additional layout adjustments (optional)
    fig.update_layout(
        title=title,
        xaxis=dict(side="bottom"),
    )

    fig.show()


In [41]:
for player in dealer_opens:
	plot_range(dealer_opens[player], title=f"{player.username}'s Opening Range from Dealer")